In [56]:
from pyrosm.data import sources
import pyrosm
from collections import Counter, defaultdict
import json
import pandas as pd
import ast
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
from skmultilearn.model_selection import iterative_train_test_split
import pickle
import random

In [57]:
def parse_tags(val):
    if pd.isna(val) or val in ["None", "nan", None]:
        return {}
    try:
        return ast.literal_eval(val) if isinstance(val, str) else val
    
    except json.JSONDecodeError:
        return {}

def create_tag_lists(pois, n):
    tag_freq = defaultdict(int)
    #print(tag_freq)



    for idx, row in pois.iterrows(): 
        tags = row.get("tags", {})
        #print(f"Row {idx}, Tags: {tags}, Type: {type(tags)}")  # Check type of tags

        if isinstance(tags, dict) and tags:
            for tag_key in tags:
                tag_freq[tag_key] += 1


    # print(tag_freq)
    all_tags = list(tag_freq.keys())
    all_good_tags = [tag for tag in tag_freq if tag_freq[tag] > n]
    tag2idx = {tag: i for i, tag in enumerate(all_good_tags)}
    idx2tag = {i: tag for tag, i in tag2idx.items()}

    tag_freq = dict(sorted(tag_freq.items(), key = lambda x: x[1], reverse = True))
    print(f"All tags sorted by frequency: {tag_freq}")
    print(f" All allowed tags: {all_good_tags}")
    # print(tag2idx)
    # print(idx2tag)
    print(f"Len all tags: {len(all_tags)}, Len good tags: {len(all_good_tags)}")
    #print("\n Returning (all_good_tags, tag2idx)")


    return (all_good_tags, tag2idx)

def tags_to_vec(tag_dict, tag2idx):
    vector = np.zeros(len(tag2idx), dtype= np.float32)
    if isinstance(tag_dict, dict):
        for tag in tag_dict:
            if tag in tag2idx:
                vector[tag2idx[tag]] = 1
    return vector

In [58]:
fp = pyrosm.get_data("Groningen")
osm = pyrosm.OSM(fp)
pois = osm.get_pois()
pois["tags"] = pois["tags"].apply(parse_tags) #only need to run it once

In [59]:
# Keep only rows where 'tags' is a non-empty dictionary
pois = pois[pois["tags"].apply(lambda x: isinstance(x, dict) and len(x) > 2)]


In [60]:
final_train, final_test = train_test_split(pois, test_size=0.2)

test_questions = []
test_answers = []
for i in final_test['tags']:
    inew = list(i.keys())
    idx = random.randint(0, len(inew)-1)
    test_answers.append(inew[idx])
    test_questions.append(inew[:idx] + inew[idx+1:])

print("q-a", len(test_questions), len(test_answers))

with open('trainingset', 'wb') as fp:
    pickle.dump(final_train, fp)
with open('testset_questions', 'wb') as fb:
    pickle.dump(test_questions, fb)
with open('testset_answer', 'wb') as fq:
    pickle.dump(test_answers, fq)

q-a 941 941


In [61]:
with open('trainingset', 'rb') as fp:
    trainingloaded = pickle.load(fp)

print(type(trainingloaded))
print(type(pois['tags']))

<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'pandas.core.series.Series'>
